# Summary

# Imports

In [ ]:
import concurrent.futures
import itertools
import importlib
import multiprocessing
import os
import os.path as op
import pickle
import subprocess
import sys
import tempfile
from functools import partial
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns
import sqlalchemy as sa
from scipy import stats

from kmtools import py_tools, sequence_tools

In [ ]:
%matplotlib inline

In [ ]:
pd.set_option("max_columns", 100)

In [ ]:
SRC_PATH = Path.cwd().joinpath('..', 'src').resolve(strict=True)

if SRC_PATH.as_posix() not in sys.path:
    sys.path.insert(0, SRC_PATH.as_posix())

import helper
importlib.reload(helper)

# Parameters

In [ ]:
NOTEBOOK_PATH = Path('validation_protherm_dataset')
NOTEBOOK_PATH

In [ ]:
OUTPUT_PATH = Path(os.getenv('OUTPUT_DIR', NOTEBOOK_PATH.name)).resolve()
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH

In [ ]:
proc = subprocess.run(["git", "rev-parse", "--short", "HEAD"], stdout=subprocess.PIPE)
GIT_REV = proc.stdout.decode().strip()
GIT_REV

In [ ]:
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")
NETWORK_NAME = os.getenv("CI_COMMIT_SHA")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

TASK_ID, TASK_COUNT, NETWORK_NAME

In [ ]:
DEBUG = "CI" not in os.environ    
DEBUG

In [ ]:
if DEBUG:
    NETWORK_NAME = "dcn_old_0,6bbf5b792c30570b8ab1a4c1b3426cdc6ad84446"
else:
    assert NETWORK_NAME is not None
    
NETWORK_NAME

In [ ]:
# if DEBUG:
#     %load_ext autoreload
#     %autoreload 2

# `DATAPKG`

In [ ]:
DATAPKG = {}

In [ ]:
DATAPKG['uniparc-domain-wstructure'] = (
    Path(os.environ['DATAPKG_OUTPUT_DIR'])
    .joinpath("uniparc-domain-wstructure", "master")
)

In [ ]:
DATAPKG['adjacency_net_v2'] = (
    Path(os.environ['DATAPKG_OUTPUT_DIR'])
    .joinpath("adjacency-net-v2", "master")
)

# Load data

In [ ]:
input_file = (
    DATAPKG['adjacency_net_v2']
    .joinpath("protherm_dataset", "protherm_validaton_dataset.parquet")
    .resolve(strict=True)
)
input_file

In [ ]:
input_df = pq.read_table(input_file).to_pandas()
input_df.head(2)

# Run network

In [ ]:
%run trained_networks.ipynb

In [ ]:
def mutate_sequence(row):
    sequence = row['sequence']
    wt = row['qseq_mutation'][0]
    pos = int(row['qseq_mutation'][1:-1])
    mut = row['qseq_mutation'][-1]
    sequence_mut = sequence[:pos - 1] + mut + sequence[pos:]
    assert len(sequence) == len(sequence_mut)
    return sequence_mut

In [ ]:
input_df['sequence'] = input_df['qseq']
input_df['sequence_mut'] = input_df.apply(mutate_sequence, axis=1)
input_df['adjacency_idx_1'] = input_df['residue_idx_1_corrected']
input_df['adjacency_idx_2'] = input_df['residue_idx_2_corrected']

In [ ]:
for network_name in NETWORK_NAME.split(','):
    input_df[f'{network_name}_wt'] = helper.predict_with_network(
        input_df[['sequence', 'adjacency_idx_1', 'adjacency_idx_2']].copy(),
        network_state=TRAINED_NETWORKS[network_name]['network_state'],
        network_info=TRAINED_NETWORKS[network_name]['network_info'],
    )
    input_df[f'{network_name}_mut'] = helper.predict_with_network(
        input_df[['sequence_mut', 'adjacency_idx_1', 'adjacency_idx_2']]
        .rename(columns={'sequence_mut': 'sequence'}).copy(),
        network_state=TRAINED_NETWORKS[network_name]['network_state'],
        network_info=TRAINED_NETWORKS[network_name]['network_info'],
    )

In [ ]:
for network_name in NETWORK_NAME.split(','):
    input_df[f'{network_name}_change'] = (
        input_df[f'{network_name}_mut'] -
        input_df[f'{network_name}_wt']
    )

## Save to cache

In [ ]:
table = pa.Table.from_pandas(input_df, preserve_index=True)
pq.write_table(
    table,
    OUTPUT_PATH.joinpath(f"{NETWORK_NAME}_dataset.parquet"),
    version='2.0',
    flavor='spark',
)

# Analyze

In [ ]:
data = [
    ('Provean', 0.25, None),
    ('FoldX', 0.48, None),
    ('ELASPIC', 0.54, None),
]

for network_name in NETWORK_NAME.split(','):
    corr, pvalue = stats.spearmanr(
        input_df[f'{network_name}_change'],
        -input_df['ddg_exp']
    )
    data.append((network_name, corr, pvalue))

In [ ]:
cmap = plt.get_cmap("Set1")

feature_names = {}

df = pd.DataFrame(data, columns=['feature', 'correlation', 'pvalue'])
df

# fg, axs = plt.subplots(figsize=(6, 3))

with plt.rc_context(rc={'figure.figsize': (6, 4), 'font.size': 13}):
    x = np.arange(len(df))
    c = [cmap(2) if f in NETWORK_NAME.split(',') else cmap(1) for f in df['feature']]
    plt.bar(x, df['correlation'], color=c)
    plt.xticks(x, [feature_names.get(f, f[:7]) for f in df['feature'].values], rotation=45)
    plt.ylim(0.4, 1)
    plt.ylabel("Correlation")
    plt.title("Predicting Protherm ΔΔG")
    plt.ylim(0, 1)

    plt.tight_layout()
    plt.savefig(OUTPUT_PATH.joinpath(f"{NETWORK_NAME}_protherm_correlations.png"), dpi=300, bbox_inches="tight")
    plt.savefig(OUTPUT_PATH.joinpath(f"{NETWORK_NAME}_protherm_correlations.pdf"), bbox_inches="tight")